In [1]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

In [2]:
compileString("""
program p
    var x, y, z : boolean
    x := false
    y := ¬ ¬ ¬  x
""", target = 'riscv')

	.data
	.text
	.globl main
main:	
	jal ra, p
	addi a0, zero, 0
	addi a7, zero, 93
	scall
	.globl p
p:	
	addi sp, sp, -32
	sw ra, 28(sp)
	sw s0, 24(sp)
	addi s0, sp, 32
	sw zero, -12(s0)
	lw s10, -12(s0)
L1:	
L2:	
	sub s5, s10, zero
	sltiu s5, s5, 1
	sw s5, -16(s0)
	lw ra, 28(sp)
	lw s0, 24(sp)
	addi sp, sp, 32
	ret


Multiple Indentations

In [ ]:
compileString("""
procedure q()
  var b: boolean
    b := true
    if b then write(3)
    else write(5)
    if ¬b then write(5)
    else if b then write(7)
    else write(9)
    while b do
      if b then
        b := false; write(1)
program p
  if 3 > 4 then writeln() else
    q()
  write(1);writeln()
  if 3 < 4 then writeln() else
    q()
  write(2);writeln()
  if 3 ≥ 4 then writeln() else
    q()
  write(3);writeln()
  if 3 ≤ 4 then writeln() else
    q()
  write(4)
""", "AssemblyFiles/RISCV-other1.s", target = 'riscv')
# 3711
#
# 2
# 3713
#
# 4

In [ ]:
!python3 -m riscemu AssemblyFiles/RISCV-other1.s

Booleans and conditions

In [ ]:
compileString("""
program p
  const five = 5
  const seven = 7
  const always = true
  const never = false
  var x, y, z: integer
  var b, t, f: boolean
    x := seven; y := 9; z := 11; t := true; f := false
    if true then write(7) else write(9)    {writes 7}
    if false then write(7) else write(9)   {writes 9}
    if t then write(7) else write(9)       {writes 7}
    if f then write(7) else write(9)       {writes 9}
    if ¬ t then write(7) else write(9)     {writes 9}
    if ¬ f then write(7) else write(9)     {writes 7}
    if t or t then write(7) else write(9)  {writes 7}
    if t or f then write(7) else write(9)  {writes 7}
    if f or t then write(7) else write(9)  {writes 7}
    if f or f then write(7) else write(9)  {writes 9}
    if t and t then write(7) else write(9) {writes 7}
    if t and f then write(7) else write(9) {writes 9}
    if f and t then write(7) else write(9) {writes 9}
    if f and f then write(7) else write(9) {writes 9}
    writeln()
    b := true
    if b then write(3) else write(5) {writes 3}
    b := false
    if b then write(3) else write(5) {writes 5}
    b := x < y
    if b then write(x) else write(y) {writes 7}
    b := (x > y) or t
    if b then write(3) else write(5) {writes 3}
    b := (x > y) or f
    if b then write(3) else write(5) {writes 5}
    b := (x = y) or (x > y)
    if b then write(3) else write(5) {writes 5}
    b := (x = y) or (x < y)
    if b then write(3) else write(5) {writes 3}
    b := f and (x ≥ y)
    if b then write(3) else write(5) {writes 5}
    writeln()
    while y > 3 do                   {writes 9, 8, 7, 6, 5, 4}
      write(y); y := y - 1
    write(y); writeln()              {writes 3}
    if ¬(x < y) and t then
      write(x)                       {writes 7}
""", 'AssemblyFiles/RISCV-other2.s', target = 'riscv')
# 79799777797999
# 35735535
# 9876543
# 7

In [ ]:
!python3 -m riscemu AssemblyFiles/RISCV-other2.s

Constant Folding, Local & Global Variables

In [ ]:
compileString("""
const seven = (9 mod 3 + 5 × 3) div 2
type int = integer
var x, y: integer
procedure q()
  const sotrue = true and true
  const sofalse = false and true
  const alsotrue = false or true
  const alsofalse = false or false
  var x: int
    x := 3
    if sotrue then y := x else y := seven
    write(y) {writes 3}
    if sofalse then y := x else y := seven
    write(y) {writes 7}
    if alsotrue then y := x else y := seven
    write(y) {writes 3}
    if alsofalse then y := x else y := seven
    write(y) {writes 7}
    if ¬(true or false) then write(5) else write(9)
program p
  x := 7; q(); write(x) {writes 7}
""", 'AssemblyFiles/RISCV-other3.s', target = 'riscv')
#373797

In [ ]:
!python3 -m riscemu AssemblyFiles/RISCV-other3.s

Procedures

In [ ]:
compileString("""
var g: integer          {global variable}
procedure q(v: integer) {value parameter}
  var l: integer        {local variable}
    l := 9
    if l > v then write(l)
    else write(g)
program p
  g := 5; q(7)          {writes 9}
""", 'AssemblyFiles/RISCV-other4.s', target = 'riscv')
#9

In [ ]:
!python3 -m riscemu AssemblyFiles/RISCV-other4.s

Nested procedures

In [ ]:
compileString("""
var b: boolean
program p
  var i: integer
  procedure q(z: boolean)
    var y: integer
    procedure r(x: integer)
      b := false; x := x + 1
    z := true; y := 4; r(i)
    write(y); writeln()
  q(b)
""", 'AssemblyFiles/RISCV-other5.s', target = 'riscv')

In [ ]:
!python3 -m riscemu AssemblyFiles/RISCV-other5.s

Input and output

In [ ]:
#read not working!!!
compileString("""
program p
  var x: integer
    x ← read()
    x := 3 × x
    write(x)
    writeln()
    writeln()
    write(x × 5)
""", 'AssemblyFiles/RISCV-other6.s', target = 'riscv')

In [ ]:
#!python3 -m riscemu AssemblyFiles/RISCV-other6.s

Two-dimensional Array

In [ ]:
compileString("""
type R = integer
type S = [1..11] → R
type T = [3..9] → S
var x: T
var y: integer
var b: boolean
program p
    var i, j : integer
    i := 3
    while i < 10 do
        j := 1
        while j < 12 do
            x[i][j] := (i + j) mod 10
            write(x[i][j])
            j := j + 1
        i := i + 1
        writeln()
    i := 3
    while i < 10 do
        j := 1
        while j < 12 do
            write(x[i][j])
            j := j + 1
        i := i + 1
        writeln()
""", 'AssemblyFiles/RISCV-other7.s', target = 'riscv')

In [ ]:
!python3 -m riscemu AssemblyFiles/RISCV-other7.s

Parameter Passing

In [ ]:
compileString("""
type T = [1..10] → integer
var x: integer
var z: T
procedure q({-4($sp)} a: integer {4($fp)}, {-8($sp)} b: integer {($fp)})
  var y: integer {-12($fp)}
    y := a; write(y); writeln() {writes 7}
    a := b; write(x); write(a); writeln() {writes 5, 5}
    b := y; write(b); write(x); writeln() {writes 7, 5}
    write(y); writeln() {writes 7}
    write(z[4]) {writes 0}
program p
  x := 5; q(7, x)
""", 'AssemblyFiles/RISCV-other8.s', target = 'riscv')
# 7
# 55
# 75
# 7
# 0

In [ ]:
!python3 -m riscemu AssemblyFiles/RISCV-other8.s